## 將所有連結存成文字檔

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string  


#################for db
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    #cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(`class`,url,newsRaw,source)values(%s,%s,%s,%s)",[category,url,newsRaw,source])
    db.commit()
def get_newsRaw(db,url):
    cur = db.cursor()
    cur.execute("select newsRaw from newsRaw where url = %s",[url])
    return cur.fetchone()
def close(db):
    db.close()
#################for db


#################Exception Handler
def EH(breakpoint,newsDate=None,category=None,aLink=None):            
    f=open("ETTodayException.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News Date: " + newsDate + "\n",
        "\t -News Category: " + category + "\n",
        "\t -News url: " + aLink + "\n",
        "\t -BreakPoint: "+ breakpoint + "\n",
        "\t -Exception Traceback: \n", 
        str(traceback.format_exc()) + "\n"
    ]
    f.writelines(ExList)
    f.close
#################Exception Handler

#################get_all_pagelink
sentinel = object()
def get_all_pagelink(startday=(str(date.today()-timedelta(1))),endday=sentinel,category=["政治","財經","國際","焦點"],sleep_time=0):
    
    f=open("libtime.text")
    if endday is sentinel:
        endday=startday
    
    def convert_day(d):
        return datetime.strptime(d, '%Y%m%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)

    assert ed >= sd, "endday must after startday"
    
    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y%m%d')))

    url_all =[]
    for newsDay in ds:
        url_dic = {"政治":"http://news.ltn.com.tw/newspaper/politics/%s"%newsDay,
                    "財經":"http://news.ltn.com.tw/newspaper/business/%s"%newsDay,
                    "國際":"http://news.ltn.com.tw/newspaper/world/%s"%newsDay,
                    "社會":"http://news.ltn.com.tw/newspaper/society/%s"%newsDay,
                    "焦點":"http://news.ltn.com.tw/newspaper/focus/%s"%newsDay,
                    "地方":"http://news.ltn.com.tw/newspaper/local/%s"%newsDay,
                    "娛樂":"http://news.ltn.com.tw/newspaper/entertainment/%s"%newsDay,
                   }
        for cat in category:
            try:
                url_all.append(url_dic[str(cat)])
            except:
                print newsDay, cat, url_dic[str(cat)], "\n"
                print traceback.format_exc() + "\n"
    
    aLink_all=[]
    for cat in url_all:
        try:
            res = requests.get(cat)
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            p = soup.select('.tit')[0].text.encode('utf-8')
            print type(p)
            b="".join(p.split("　"))
            c="".join(b.split(" "))
            page = re.search(".*共有(\d{1,4})筆.*",c).group(1)
            pageCount = int(math.ceil(float(page)/20))
            soups = [soup]
        except:
            print cat+ "\n"
            print traceback.format_exc()+ "\n"
        if pageCount >1:
            for i in xrange(2,pageCount+1):
                curl = str(cat)+"?page=%s"%i
                res = requests.get(curl)
                res.encoding="utf-8"
                soups.append(BeautifulSoup(res.text))

        for soup in soups:         
            for block in soup.select(".picword"):
                try:
                    aLink='http://news.ltn.com.tw'+block['href']
                    aLink_all.append(aLink)
                    f.write(aLink+"\n")
                except:
                    print cat
                    raise
                finally:
                    f.close()
        time.sleep(sleep_time)
    f.close()
    return aLink_all    
#################get_all_pagelink


## 寫進raw data

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string  


#################for db
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    #cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(`class`,url,newsRaw,source)values(%s,%s,%s,%s)",[category,url,newsRaw,source])
    db.commit()
def get_newsRaw(db,url):
    cur = db.cursor()
    cur.execute("select newsRaw from newsRaw where url = %s",[url])
    return cur.fetchone()
def close(db):
    db.close()
#################for db


#################Exception Handler
def EH(breakpoint,newsDate=None,category=None,aLink=None):            
    f=open("ETTodayException.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News Date: " + newsDate + "\n",
        "\t -News Category: " + category + "\n",
        "\t -News url: " + aLink + "\n",
        "\t -BreakPoint: "+ breakpoint + "\n",
        "\t -Exception Traceback: \n", 
        str(traceback.format_exc()) + "\n"
    ]
    f.writelines(ExList)
    f.close
#################Exception Handler

#################crawl_all_rawpage_and_insert_into_db
def all_rawpage(aLink_all,db="practice",passwd="",port="localhost",user="root"):
    
        #driver = webdriver.Firefox()
        #driver = webdriver.Chrome()
        driver = webdriver.PhantomJS()
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
    
    for aLink in aLink_all:
        #########rawpage breakpoint
        try:
            driver.get(aLink)
            WebDriverWait(driver, 30).until(
                    EC.visibility_of_element_located((By.ID, "fbComments"))
                )
            html = driver.page_source
            cont = str(html.encode('utf-8'))
            conn=connect(db,passwd,port,user)
            insert_newsRaw(conn,aLink,cont,source="LibertyTime")
        except Exception as e:
            EH('rawpage',newsDate=None,category=None,aLink=None) 
            print "rawpage not write in DB \n", traceback.format_exc(),aLink,"\n"
            continue
        finally:
            driver.quit()
            close(conn)
        #########rawpage breakpoint                
#################crawl_all_rawpage_and_insert_into_db

## 寫進乾淨的資料

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string  


#################for db
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    #cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(`class`,url,newsRaw,source)values(%s,%s,%s,%s)",[category,url,newsRaw,source])
    db.commit()
def get_newsRaw(db,url):
    cur = db.cursor()
    cur.execute("select newsRaw from newsRaw where url = %s",[url])
    return cur.fetchone()
def close(db):
    db.close()
#################for db


#################Exception Handler
def EH(breakpoint,newsDate=None,category=None,aLink=None):            
    f=open("ETTodayException.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News Date: " + newsDate + "\n",
        "\t -News Category: " + category + "\n",
        "\t -News url: " + aLink + "\n",
        "\t -BreakPoint: "+ breakpoint + "\n",
        "\t -Exception Traceback: \n", 
        str(traceback.format_exc()) + "\n"
    ]
    f.writelines(ExList)
    f.close
#################Exception Handler

#################parse_page_and_insert_into_db
hit = None
sentinel = object()
def parse_page(aLink_all,db="news",passwd="",port="localhost",user="root"):
    #########retrieve_rawcontent_from_db
    try:
        conn=connect(db,passwd,port,user)
        cur=conn.cursor()
        cur.execute("select newsRaw,url from newsRaw;")
        allRaw=cur.fetchall()
    except:
        print "retrieve rawcontent not succeed...\n", traceback.format_exc(),"\n"
    #########retrieve_rawcontent_from_db
    
    for raw in allRaw:
        contRaw=raw[0]
        soup=BeautifulSoup(contRaw)
        aLink=raw[1]
        content = soup.select('.content')[0]
        newstext = content.select('#newstext')[0]
        #########category breakpoint
        try:
            category=content.select('.guide a')[1].text.encode('utf-8')
        except:
            EH(breakpoint="category",newsDate=newsDate,category=category,aLink=aLink)
        #########category breakpoint

        #########title breakpoint
        try:
            title=content.select('h1')[0].text.encode('utf-8')
        except:
            EH(breakpoint="title",newsDate=newsDate,category=category,aLink=aLink)
        #########title breakpoint

        ########Date&Time breakpoint
        try:
            newsDate=content.select('#newstext span')[0].text.encode('utf-8')
        #newsTime=timeRegex.group(3)
        except:
            EH(breakpoint="Date&Time",newsDate=newsDate,category=category,aLink=aLink)
        ########Date&Time breakpoint

        #########content breakpoint
        try:
            
            try:
                for tag in newstext.select('script'):
                    tag.decompose()
                for tag in newstext.select('#newsad'):
                    tag.decompose()                       
            except:
                pass 
            content=newstext.text
        except:
            EH(breakpoint="content",newsDate=newsDate,category=category,aLink=aLink)
        #########content breakpoint        

        #########journist breakpoint
        jor = ((newstext.select('p')[0].text.encode('utf-8')).split('／')[0].decode('utf-8'))
        try:
            journalist=re.search(u'.*(?:編譯|記者|◎)([^）]*)', jor).group(1).strip()
        except:
            EH(breakpoint='journalist',newsDate=newsDate,category=category,aLink=aLink) 
            print "jor: ",aLink," WTF"
        #########journist breakpoint

        '''#########comment breakpoint
        comments=[]
        try:
            commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
            commLink1= commLink0.select(' span > iframe')[0]['src']
            commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
            resComm = requests.get(commLink)
            resComm.encoding = "utf-8"
            soupComm = BeautifulSoup(resComm.text)      
            commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
            for comm in commentBlock: 
                dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                commentTime=dt.group(4)
                commenter=comm.select('.profileName')[0].text
                commenterUrl=comm.select('.profileName')[0]['href']
                comment=comm.select('.postText')[0].text
                comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
        except Exception as e:
            EH('comment',newsDate=None,category=None,aLink=None) 
        #########comment breakpoint'''

        #################connect to db
        #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
        try:
            #insert_news(conn,category,title,newsDate,newsTime,journalist,content,hit,aLink)
            insert_news(conn,category,title,newsDate,journalist,content,hit,aLink)
            #for c in comments:
            #    insert_comments(conn,c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5])
        except:
            print "something not write in DB \n", traceback.format_exc(),aLink,"\n"
        finally:
            close(conn)
        #################connect to db
#################parse_page_and_insert_into_db   